In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

from operator import add
class LazySegmentTree:
  def __init__(self, data, f=lambda x, y: x+y, fl=lambda i, x, cnt: i+x*cnt, fll=lambda x, y: x+y, default=0, ldefault=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.default = default
    self.ldefault = ldefault
    self.len = len(data)
    self.L = [default] * self.len + data
    self.lazy = [ldefault] * self.len
    self.cnt = [0] * self.len + [1] * self.len

    for i in reversed(range(self.len)):
      self.L[i] = self.f(self.L[i*2], self.L[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _apply(self, i) :
    if self.lazy[i] == self.ldefault: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.ldefault 

  def _propagate(self, i):
    for h in reversed(range(1, self.len.bit_length()+1)) :
      idx = i >> h

      if idx == self.default : continue
      self._apply(idx)

  def _push(self, idx, val) :
    self.L[idx] = self.fl(self.L[idx], val, self.cnt[idx])
    if idx < self.len :
      self.lazy[idx] = self.fll(self.lazy[idx], val)

  def _build(self, idx) :
    while idx :
      if idx < self.len :
        self._apply(idx)
        self.L[idx] = self.f(self.L[idx*2], self.L[idx*2+1])
      idx >>= 1

  def query(self, _s, _e) : #[s, e]
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    res = self.default
    while s < e :
      if s & 1 :
        res = self.f(res, self.L[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.L[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, _s, _e, x):
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e)

def sol() :
  N = int(input())
  L = [tuple(map(int, input().split())) for _ in range(N)]

  cmp = set() # compress
  for x1, _, y1, _ in L :
    cmp.add(x1)
    cmp.add(y1)
  
  cmp = sorted(list(cmp))
  cmp2 = {v: i for i, v in enumerate(cmp)}
  
  L2 = []
  for i, (_x1, y1, _x2, y2) in enumerate(L) :
    x1 = cmp2[_x1]
    x2 = cmp2[_x2]
    L2.append((x1, x2, y1, 1, i))
    L2.append((x1, x2, y2, -1, i))
  L.sort(key=lambda x: (x[2], -x[3]))

  lst = LazySegmentTree([0] * len(cmp), f=max, fl=lambda i, x, cnt: i+x)
  ans = 0
  
sol()

### 풀이
- 좌표압축이 필요하지 않은 것 같다. 애초에 x좌표가 왜 필요한지 계속 의문이였는데 필요하지 않은게 맞는가보다.
- https://koosaga.com/220
- https://grace100000.tistory.com/4
- 멘탈 채우고 재도전